In [79]:
import datetime
import numpy as np
import pandas as pd
import urllib2
import json
from pandas.io.json import json_normalize

import db_service
import config


def get_sites_dict():
    sites = db_service.get_sites()
    sites = sites[['idbldsite','sname','latitude','longitude']]
    sites = sites.set_index('idbldsite').T
    sites_dict =  sites.to_dict()
    return sites_dict

def get_weather(lat,lon):
    response = urllib2.urlopen(config.WEATHER_URL+"lat="+lat+"&lon="+lon)
    data = json.loads(response.read())
    df_weather = json_normalize(data['LocationWeather']['forecast']['days'])
    df_weather.date = pd.to_datetime(df_weather.date)
    last_update = json_normalize(data['LocationWeather']['latestobservation']['latests'][0])['dateTime'].values[0]
    
    df_weather['lastUpdate']= pd.to_datetime(last_update)
    return df_weather
    

def get_weather_forecasts():
    sites_dict = get_sites_dict()
    df_weather = pd.DataFrame()
    
    for site_id in sites_dict:
        sname = sites_dict[site_id]['sname']
        lat = str(sites_dict[site_id]['latitude'])
        lon = str(sites_dict[site_id]['longitude'])
        
        df_weather_site = get_weather(lat,lon)
        df_weather_site['idbldsite']= site_id
        df_weather_site['sname']= sname
        df_weather = pd.concat([df_weather,df_weather_site],axis=0)
        df_weather.columns = ['idbldsite', 'sname','date', 'ne', 'tn', 'tx', 'ww', 'lastUpdate']
        df_weather.to_csv('data/weather_forecasts.csv',index=0)
        

In [80]:
get_weather_forecasts()

In [76]:
df_weather = pd.read_csv('data/weather_forecasts.csv')

In [78]:
df_weather.columns.tolist()

['date', 'ne', 'tn', 'tx', 'ww', 'lastUpdate', 'idbldsite', 'sname']